In [8]:
import os
from os import path
import re
import urllib
import urllib.request
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup as soup
import requests
from bs4 import BeautifulSoup
from google.cloud import speech_v1p1beta1 as speech
import speech_recognition as sr
from pydub import AudioSegment
import subprocess


In [9]:
url = 'https://peterattiamd.com/ama47/'
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

title = page_soup.find("title")
print(title)

<title>#254 - AMA #47: Cold therapy: pros, cons, and its impact on longevity - Peter Attia</title>


In [10]:
print(page_soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="en-US">
 <head>
  <script type="text/javascript">
   /* <![CDATA[ */
 var gform;gform||(document.addEventListener("gform_main_scripts_loaded",function(){gform.scriptsLoaded=!0}),window.addEventListener("DOMContentLoaded",function(){gform.domLoaded=!0}),gform={domLoaded:!1,scriptsLoaded:!1,initializeOnLoaded:function(o){gform.domLoaded&&gform.scriptsLoaded?o():!gform.domLoaded&&gform.scriptsLoaded?window.addEventListener("DOMContentLoaded",o):document.addEventListener("gform_main_scripts_loaded",o)},hooks:{action:{},filter:{}},addAction:function(o,n,r,t){gform.addHook("action",o,n,r,t)},addFilter:function(o,n,r,t){gform.addHook("filter",o,n,r,t)},doAction:function(o){gform.doHook("action",o,arguments)},applyFilters:function(o){return gform.doHook("filter",o,arguments)},removeAction:function(o,n){gform.removeHook("action",o,n)},removeFilter:function(o,n,r){gform.removeHook("filter",o,n,r)},addHook:function(o,n,r,t,i){null==gform.hooks[o][n]&&(gfo

In [11]:
links = []
text = page_soup.prettify()

text = text.replace("\\/", "/")
text = text.replace(" ","_")
regex = re.findall(r'http.*?mp3', text)

sum = ''
for e in regex:
    sum += e
    
res = sum.split(',')

links = [re.findall(r'http.*?mp3', i) for i in res]

unpacked = [i[0] for i in links if i != []]
print(unpacked)
#re.findall(r'https?:\/\/.*\.mp3$', text)


['https://traffic.libsyn.com/secure/peterattiadrive/EP.254_-_AMA_47_SNEEK_PEEK.mp3', 'https://traffic.libsyn.com/secure/peterattiadrive/EP.254_-_AMA_47_SNEEK_PEEK.mp3']


In [12]:
unique_list = list(set(unpacked))

folder = "audios/"

if not os.path.exists(folder):
    os.makedirs(folder)

filenames = []

for url in unique_list:
    filenames.append(url.split('/')[-1])

for ind, ele in enumerate(unique_list):
    url = f'{folder}{filenames[ind]}'
    urlretrieve(ele, url)
    print(url)


audios/EP.254_-_AMA_47_SNEEK_PEEK.mp3


In [13]:
r = sr.Recognizer()

audio_file = sr.AudioFile('audios/'+'EP.254_-_AMA_47_SNEEK_PEEK.wav')

with audio_file as source:
    audio = r.record(source, duration=40)

"""    
with audio_file as source:
    audio = r.record(source)
"""
text = r.recognize_google(audio)
text

"hey everyone welcome to a sneak peek ask me anything or Uma episode of The Drive podcast I'm your host Peter Attia at the end of this short episode I'll explain how you can access the AMA episodes in full along with a ton of other membership benefits we've created or you can learn more now by going to Peter Attia md.com forward slash subscribe so without further delay here's today's sneak peek of the ask me anything episode"

In [14]:
# Function to transcribe audio to text using Google Cloud Speech-to-Text
def transcribe_audio(audio_url):
    client = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri=audio_url)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.OGG_OPUS,
        language_code="en-US",
    )
    response = client.recognize(config=config, audio=audio)
    transcripts = []
    for result in response.results:
        transcripts.append(result.alternatives[0].transcript)
    return transcripts

# URL of the webpage containing the audio
url = "https://content.libsyn.com/p/d/1/b/d1b65ad733f9441c/EP.254_-_AMA_47_SNEEK_PEEK.mp3?c_id=151844049&cs_id=151844049&response-content-type=audio%2Fmpeg&Expires=1714302958&Signature=ZoU1VHP2md8aJSWHfUdjpLK3~iZ1II0G7hhFxN0wPC50NGUw7jcNL3O4zwbJsgzeQdWcakT2rsAe-qO-VJqXD1jkDX9glPox~inBEJibxcwlH7jhb3fyPITfa8BnYBlmkl~6yiI6oT1VejMRWV1SGt9ed-YvAhMW3japlpRd2cBTGB3ioxDuDE7JWB5pJ6YvF7hmTd6oq2a94rycanbTI5kpBwcb3O6eFl5XI4iY0hj-c9bfjbFeangw5J7ZYaGh~B5BvdSRvN0EQVcLf5KTy4tF~LI8a0FMKcqf-I5BFkLGUcACwJnINNkgFBefmZ3LvHyig2V4lwH4FDHdY-IOdg__&Key-Pair-Id=K1YS7LZGUP96OI" 
# "https:\/\/traffic.libsyn.com\/secure\/peterattiadrive\/EP.254_-_AMA_47_SNEEK_PEEK.mp3"
# "https://content.libsyn.com/p/d/1/b/d1b65ad733f9441c/EP.254_-_AMA_47_SNEEK_PEEK.mp3?c_id=151844049&cs_id=151844049&response-content-type=audio%2Fmpeg&Expires=1714302958&Signature=ZoU1VHP2md8aJSWHfUdjpLK3~iZ1II0G7hhFxN0wPC50NGUw7jcNL3O4zwbJsgzeQdWcakT2rsAe-qO-VJqXD1jkDX9glPox~inBEJibxcwlH7jhb3fyPITfa8BnYBlmkl~6yiI6oT1VejMRWV1SGt9ed-YvAhMW3japlpRd2cBTGB3ioxDuDE7JWB5pJ6YvF7hmTd6oq2a94rycanbTI5kpBwcb3O6eFl5XI4iY0hj-c9bfjbFeangw5J7ZYaGh~B5BvdSRvN0EQVcLf5KTy4tF~LI8a0FMKcqf-I5BFkLGUcACwJnINNkgFBefmZ3LvHyig2V4lwH4FDHdY-IOdg__&Key-Pair-Id=K1YS7LZGUP96OI" 
#"https://peterattiamd.com/ama47/"

# Scrape the audio URL
audio_url = scrape_audio_url(url)
if audio_url:
    print("Audio URL found:", audio_url)
    # Transcribe the audio to text
    transcripts = transcribe_audio(audio_url)
    print("Transcription:")
    for transcript in transcripts:
        print(transcript)
else:
    print("No audio found on the webpage.")

NameError: name 'scrape_audio_url' is not defined

In [ ]:
test_mp3 = filenames[0]

client = speech.SpeechClient()

# Configure audio settings
config = {
    "language_code": "en-US",
}
audio = {"uri": test_mp3}

# Perform the transcription
response = client.long_running_recognize(config=config, audio=audio)
operation = response.operation
operation_result = operation.result()

# Parse the transcription results
transcription = ""
for result in operation_result.results:
    for alternative in result.alternatives:
        transcription += alternative.transcript + "\n"

# Provide the path to your audio file
audio_file_path = test_mp3

# Call the transcribe_audio function
transcription = transcribe_audio(audio_file_path)
print("Transcription:")
print(transcription)


## Working WAV+transcript

In [15]:
r = sr.Recognizer()

audio_file = sr.AudioFile(folder+'EP.254_-_AMA_47_SNEEK_PEEK.wav')

with audio_file as source:
    audio = r.record(source, duration=40)

"""    
with audio_file as source:
    audio = r.record(source)
"""
text = r.recognize_google(audio)
print(text)

hey everyone welcome to a sneak peek ask me anything or Uma episode of The Drive podcast I'm your host Peter Attia at the end of this short episode I'll explain how you can access the AMA episodes in full along with a ton of other membership benefits we've created or you can learn more now by going to Peter Attia md.com forward slash subscribe so without further delay here's today's sneak peek of the ask me anything episode


In [ ]:
def transcribe_audio(audio_file):
    recognizer = sr.Recognizer()

    # Load audio file
    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    # Perform the transcription using CMU Sphinx
    try:
        transcription = recognizer.recognize_sphinx(audio_data)
        return transcription
    except sr.UnknownValueError:
        return "Sphinx could not understand the audio"
    except sr.RequestError as e:
        return f"Error occurred in Sphinx; {e}"


# Convert MP3 to WAV
def convert_to_wav_sox(mp3_file):
    # assign files 
    input_file = mp3_file
    output_file = input_file[:-4]+'.wav'
    
    # convert mp3 file to wav file 
    sound = AudioSegment.from_mp3(input_file)
    sound.export(output_file, format="wav")
    return output_file

In [ ]:
# Provide the path to your audio file
audio_file_path = folder+filenames[0]
print(audio_file_path)

# Convert MP3 to WAV
wav_file_path = convert_to_wav_sox(audio_file_path)


In [ ]:
# Call the transcribe_audio function
print(wav_file_path)
#transcription = transcribe_audio(wav_file_path)

print("Transcription:")
print(transcription)